In [1]:
import numpy as np
import utils as ut
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

#function to generate perturbed matrices close to original matrix
def pert_matrix(mat_in,scale,n_rnd_param): 
    #mat_in: squared matrix to be perturbed (N x N)
    #scale: added perturbation between [0,scale]
    #n_rnd_param: number of perturbed matrix generated
    
    #dimension of the problem
    N = mat_in.shape[0]
    
    #eigendecomposition
    w,v = np.linalg.eig(mat_in)
    V   = v
    W   = np.diag(w)

    #generate perturbed matrices
    A_noise = []
    for rng in range(n_rnd_param):
        V_noise = scale*np.random.rand(N,N) + V
        A_noise.append(V_noise @ W @ np.linalg.inv(V_noise))
        
    return A_noise
 

In [2]:
## Input parameters
scale = 0.1     #scale for which original is perturbed
n_rnd_param = 2 #number of perturbed matrices to generate
x0 = np.transpose(np.matrix([[0.3,0.4,0.7]]))  #starting point (value from which forecast are made)
horizon = 4    #how many steps-ahead the forecast are made
n_samples = 5  #how many forecast/trajectories are generated for EACH sampled VAR parameter

## Real values to estimate (array)
A = np.array([[.5,0,0],[.1,.1,.3],[0,.2,.3]]) #A = A1, VAR(p=1)
U = np.array([[2,0.11,0.09],[0.11,1.5,0.13],[0.09,0.13,1]])
print(U)
print(np.linalg.eigvals(A))
print(np.linalg.eigvals(U))

## Generate perturbed matrices
A_noise = pert_matrix(A,scale,n_rnd_param)
U_noise = pert_matrix(U,scale,n_rnd_param)

## Power curve
data_csv = pd.read_csv('data_power.csv') # Speed-PowerGeneration data.
xdata = data_csv['Wind speed (m/s)'].values
ydata = data_csv['Power (kW)'].values
fit_out = curve_fit(ut.PL5, xdata, ydata, maxfev=10000)
a = fit_out[0][0]
b = fit_out[0][1]
c = fit_out[0][2]
d = fit_out[0][3]
g = fit_out[0][4]

[[2.   0.11 0.09]
 [0.11 1.5  0.13]
 [0.09 0.13 1.  ]]
[ 0.46457513 -0.06457513  0.5       ]
[2.03611991 1.49939394 0.96448615]


In [7]:
## Simulation of wind speed
cap_wind = 3 # wind plant capacity (eg. 3MW)
cut_speed = 20 # Speed where generator stop working for security.

sampl_traj = np.zeros((n_samples,horizon,n_rnd_param)) #list of generated trajectories for EACH sampled VAR paremeter
for s_samp in range(n_rnd_param):
    A_s = A_noise[s_samp]
    U_s = U_noise[s_samp]
    traj_wind = ut.sim_wind(A_s,U_s,x0,horizon,n_samples)
    pow_wind = ut.power_curve(traj_wind, cap_wind, cut_speed, a, b, c, d, g)
    sampl_traj[:,:,s_samp] = np.sum(pow_wind,axis=0)
    
print(sampl_traj.shape) #n_samples,horizon,n_rnd_param
print(sampl_traj[:,:,0])
print(sampl_traj[:,:,1])

y = np.swapaxes(sampl_traj,1,2)
y = np.reshape(y,(n_samples*n_rnd_param,horizon))
print(y.shape)
print(y)

(5, 4, 2)
[[0.12139884 0.06061942 0.12123124 0.0612376 ]
 [0.18413171 0.13097145 0.12108267 0.38377883]
 [0.06073441 0.14664256 0.31757718 0.21608813]
 [0.06053257 0.06053269 0.         0.06093419]
 [0.20289829 0.06150546 0.06133804 0.        ]]
[[0.12246379 0.18229831 0.06053391 0.06053425]
 [0.06831003 0.06420185 0.06054258 0.06115063]
 [0.06837318 0.19017436 0.15290775 0.22302107]
 [0.0606376  0.06053267 0.06057061 0.12106573]
 [0.12174255 0.18325269 0.0606169  0.        ]]
(10, 4)
[[0.12139884 0.06061942 0.12123124 0.0612376 ]
 [0.12246379 0.18229831 0.06053391 0.06053425]
 [0.18413171 0.13097145 0.12108267 0.38377883]
 [0.06831003 0.06420185 0.06054258 0.06115063]
 [0.06073441 0.14664256 0.31757718 0.21608813]
 [0.06837318 0.19017436 0.15290775 0.22302107]
 [0.06053257 0.06053269 0.         0.06093419]
 [0.0606376  0.06053267 0.06057061 0.12106573]
 [0.20289829 0.06150546 0.06133804 0.        ]
 [0.12174255 0.18325269 0.0606169  0.        ]]


C:\Users\Samuel\Desktop\2018_01\IIC 3695 Intel Maquina\Tareas y Proyecto\Proyecto\Code\TAIM\utils.py:413: RuntimeWarning: covariance is not positive-semidefinite.
  samples = np.random.multivariate_normal(np.zeros(Kv),CovU,size=n_samples)
